# Router

In [34]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from operator import itemgetter
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnableBranch

OPEN_AI_APIKEY="{OPEN AI KEY}"
model = OpenAI(openai_api_key=OPEN_AI_APIKEY)

restaurant_template = """
You are a tourist guide. You know many good restaurants around the tourist destination.
You can recommend good foods and restaurants.
Here is a question:
{input}
"""

transport_template = """
You are a tourist guide. You have a lot of knowledge in public transportation.
You can provide information about public transportation to help tourists get to the tourist destination.
Here is a question:
{input}
"""

destination_template = """
You are a tourist guide. You know many good tourist places.
You can recommend good tourist places to the tourists.
Here is a question:
{input}
"""

prompt_infos = [
    {
        "name":"restaurants",
        "description":"Good for recommending restaurants around the tourist destinations",
        "prompt_template": restaurant_template
    },
    {
        "name":"transport",
        "description":"Good for guiding the transport to get the place",
        "prompt_template": transport_template
    },
    {
        "name":"destination",
        "description":"Good for recommending place to tour",
        "prompt_template": destination_template
    }
]

destination_chains = {}
for prompt_info in prompt_infos:
    name = prompt_info["name"]
    prompt = PromptTemplate.from_template(prompt_info["prompt_template"])
    chain = LLMChain(llm = model, prompt=prompt,verbose=True)
    destination_chains[name] = chain
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

default_prompt = PromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=model, prompt=default_prompt)

router_prompt = PromptTemplate(template="""
For the given question, refer to the explanations for the following categories below and select the appropriate category. 
If there is no suitable category, return the string "default".
The output should be single word 

Categories are composed of strings in the format "Category: Description", and the categories are as follows:
{destinations_str}

Question:
{input}
""",input_variables=["input"],partial_variables={"destinations_str":destinations_str})

print(destinations_str)
router_chain = {"input":itemgetter("input")} |router_prompt | model | StrOutputParser()

#branch = RunnableBranch(
#    (lambda x: "restaurants" in x['destination'].lower(), destination_chains['restaurants']),
#    (lambda x: "transport" in x['destination'].lower(), destination_chains['transport']),
#    (lambda x: "destination" in x['destination'].lower(), destination_chains['destination']),
#    default_chain,
#)

branch = RunnableBranch(
    *(
        (lambda x, name=prompt_info["name"], chain=destination_chains[name]: name in x['destination'].replace("\n", ""), chain) 
        for prompt_info in prompt_infos
    ),
    default_chain
)

full_chain = {"input":itemgetter("input"),"destination":router_chain} | branch 

input = "What is best restaurant in Seoul?"
output = full_chain.invoke({"input":input})
   
print(output['text'])

restaurants: Good for recommending restaurants around the tourist destinations
transport: Good for guiding the transport to get the place
destination: Good for recommending place to tour


> Entering new LLMChain chain...
Prompt after formatting:

You are a tourist guide. You know many good tourist places.
You can recommend good tourist places to the tourists.
Here is a question:
What is best restaurant in Seoul?


> Finished chain.

As a tourist guide, I highly recommend visiting the famous Gwangjang Market in Seoul. It is a bustling market filled with a variety of food stalls offering traditional Korean dishes such as bibimbap, tteokbokki, and mandu. Among the many options, I would say the best restaurant in Seoul is a small stall called "Myeongdong Kyoja" which specializes in handmade kalguksu (knife-cut noodles) and mandu (dumplings). The dishes are made with fresh and high-quality ingredients, and the flavors are simply delicious. The restaurant is also conveniently located near p